<a href="https://colab.research.google.com/github/Ayben06/UserClassfication/blob/main/MachineLearningProjectForUserClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the relevant libraries

# Data Preprocessing

### Importing the Database

### Removing Outliers

### Checking for Multicollinearity

### Dealing with NaN Values

### Splitting the Data

### Encoding the Data

# Creating a Logistic Regression Model

# Creating a K-Nearest Neighbors Model

# Creating a Support Vector Machines Model

# Creating a Decision Trees Model

# Creating a Random Forests Model